In [ ]:
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import tiktoken
from dotenv import load_dotenv
import wikipediaapi
import requests
from bs4 import BeautifulSoup
import tiktoken
from urllib.parse import unquote
from urllib.request import urlopen
# Cell to import necessary libraries
import re
import os
import openai
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
nltk.download('stopwords')

from langchain.chains.openai_functions import (
    create_structured_output_runnable,
    create_structured_output_chain,
)



load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)


In [ ]:

# Initialize the language model

def load_data(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

def save_data(data, filepath):
    with open(filepath, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4)

def normalize_name(name):
    return ' '.join(name.lower().strip().split())

def deduplicate_with_llm(entities):
    """Use LLM to semantically deduplicate entities based on descriptions and context."""
    deduplicated = []
    used = set()  # Track indices of entities that have been merged

    for i, current in enumerate(entities):
        if i in used:
            continue

        # Normalize and set up initial entity structure
        current['aliases'] = set([normalize_name(alias) for alias in current.get('aliases', [])] + [normalize_name(current['character'])])
        current['fragments'] = set([current['fragment']])

        for j in range(i + 1, len(entities)):
            if j in used:
                continue
            other = entities[j]

            # Normalize other entity data
            other['aliases'] = set([normalize_name(alias) for alias in other.get('aliases', [])] + [normalize_name(other['character'])])
            
            # Construct the prompt to check if entities are the same
            prompt = [
                {
                    "role": "system",
                    "content": f"Consider these entities:\n1. {current['character']} with aliases {list(current['aliases'])} and fragment: {current['fragment']}\n2. {other['character']} with aliases {list(other['aliases'])} and fragment: {other['fragment']}\nAre these descriptions of the same character?"
                }
            ]

            response = llm.invoke(prompt)  # Invoke the LLM with the prompt
            response_text = response.content  # Access the response content directly

            # Check if the response indicates the entities are the same
            if 'yes' in response_text.lower():
                current['aliases'].update(other['aliases'])
                current['fragments'].update([other['fragment']])
                used.add(j)

        # Convert sets back to lists for JSON serialization
        current['aliases'] = list(current['aliases'])
        current['fragments'] = list(current['fragments'])
        deduplicated.append(current)

    return deduplicated

# Example use of the function remains the same
# def deduplicate_with_llm(entities):
#     """ Use LLM to semantically deduplicate entities based on descriptions and context. """
#     deduplicated = []
#     used = set()  # Track indices of entities that have been merged

#     for i, current in enumerate(entities):
#         if i in used:
#             continue

#         current['aliases'] = set([normalize_name(alias) for alias in current.get('aliases', [])] + [normalize_name(current['character'])])
#         current['fragment'] = set(current.get('fragment', [current['fragment']]))

#         for j in range(i + 1, len(entities)):
#             if j in used:
#                 continue
#             other = entities[j]

#             other['aliases'] = set([normalize_name(alias) for alias in other.get('aliases', [])] + [normalize_name(other['character'])])
#             other['fragment'] = set(other.get('fragment', [other['fragment']]))

#             prompt = [
#                 {
#                     "role": "system",
#                     "content": f"""
#                     Please analyze the following character profiles:

#                     Profile 1:
#                     Name: {current['character']}
#                     Aliases: {', '.join(current['aliases'])}
#                     Key Fragments: {', '.join(current['fragment'])}

#                     Profile 2:
#                     Name: {other['character']}
#                     Aliases: {', '.join(other['aliases'])}
#                     Key Fragments: {', '.join(other['fragment'])}

#                     Considering their names, aliases, and the contexts provided by the key fragments, are these profiles describing the same individual? Evaluate their identities based on overlapping information and narrative connections.
#                     """
#                 }
#             ]

#             response = llm.invoke(prompt)  # Invoke the LLM with the prompt
#             response_text = response.content  # Access the response content directly
#             print(response_text)
#             # Check if the response indicates the entities are the same
#             if 'yes' in response_text.lower() or 'likely' in response_text.lower():
#                 current['aliases'].update(other['aliases'])
#                 current['fragment'].update(other['fragment'])
#                 used.add(j)

#         # Convert sets back to lists for JSON serialization
#         current['aliases'] = list(current['aliases'])
#         current['fragment'] = list(current['fragment'])
#         deduplicated.append(current)

#     return deduplicated


In [ ]:
# Load data
entities = load_data('data/Merged_Entity_json/Interstellar (film)/Interstellar (film)_Merged.json')

# Deduplicate entities
deduplicated_entities = deduplicate_with_llm(entities)

# Save deduplicated data
save_data(deduplicated_entities, 'data/Deduped_Entity_json/Interstellar/Interstellar_Deduped.json')
